<a href="https://colab.research.google.com/github/Nastiiasaenko/Deeplearning-Final-Project/blob/main/final_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load In-Distribution (ID) Data: Wikipedia
id_dataset = load_dataset(
    "wikipedia", "20220301.en", split="train[:1%]", trust_remote_code=True
)  # 1% for demo

# Initialize a tokenizer (e.g., BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


# Preprocess ID Data
def preprocess_id_data(dataset, tokenizer):
    texts = [entry["text"] for entry in dataset]
    tokenized = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    return tokenized


id_data = preprocess_id_data(id_dataset, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch

# Divide dataset into chunks
chunk_size = 1000  # Adjust based on memory constraints
for i in range(0, len(id_data["input_ids"]), chunk_size):
    chunk = {key: value[i : i + chunk_size] for key, value in id_data.items()}
    torch.save(chunk, f"id_data_chunk_{i // chunk_size}.pt")
    print(f"Saved chunk {i // chunk_size}")

Saved chunk 0
Saved chunk 1
Saved chunk 2
Saved chunk 3
Saved chunk 4
Saved chunk 5
Saved chunk 6
Saved chunk 7
Saved chunk 8
Saved chunk 9
Saved chunk 10
Saved chunk 11
Saved chunk 12
Saved chunk 13
Saved chunk 14
Saved chunk 15
Saved chunk 16
Saved chunk 17
Saved chunk 18
Saved chunk 19
Saved chunk 20
Saved chunk 21
Saved chunk 22
Saved chunk 23
Saved chunk 24
Saved chunk 25
Saved chunk 26
Saved chunk 27
Saved chunk 28
Saved chunk 29
Saved chunk 30
Saved chunk 31
Saved chunk 32
Saved chunk 33
Saved chunk 34
Saved chunk 35
Saved chunk 36
Saved chunk 37
Saved chunk 38
Saved chunk 39
Saved chunk 40
Saved chunk 41
Saved chunk 42
Saved chunk 43
Saved chunk 44
Saved chunk 45
Saved chunk 46
Saved chunk 47
Saved chunk 48
Saved chunk 49
Saved chunk 50
Saved chunk 51
Saved chunk 52
Saved chunk 53
Saved chunk 54
Saved chunk 55
Saved chunk 56
Saved chunk 57
Saved chunk 58
Saved chunk 59
Saved chunk 60
Saved chunk 61
Saved chunk 62
Saved chunk 63
Saved chunk 64


In [ ]:
print(f"Input IDs Shape: {id_data['input_ids'].shape}")
print(f"Attention Mask Shape: {id_data['attention_mask'].shape}")

Input IDs Shape: torch.Size([64587, 512])
Attention Mask Shape: torch.Size([64587, 512])


In [ ]:
import random


def random_word_insertion(sentence, n=3):
    words = sentence.split()
    for _ in range(n):
        words.insert(
            random.randint(0, len(words)),
            random.choice(["foobar", "xyz123", "nonsense"]),
        )
    return " ".join(words)


def generate_random_noise(num_sentences=100, length=15):
    import string

    sentences = [
        "".join(
            random.choices(
                string.ascii_letters + string.digits + string.punctuation, k=length
            )
        )
        for _ in range(num_sentences)
    ]
    return sentences


def generate_random_sentences(vocab, length=10, num_sentences=100):
    sentences = [
        " ".join(random.choices(vocab, k=length)) for _ in range(num_sentences)
    ]
    return sentences

In [ ]:
import random
import string

# Define base sentence
base_sentence = "This is a nonsensical OOD example"

# Create shuffled sentences
shuffled_sentences = [
    " ".join(random.sample(base_sentence.split(), len(base_sentence.split())))
    for _ in range(100)
]

# Create random word insertion sentences
random_word_sentences = [random_word_insertion(base_sentence, n=3) for _ in range(100)]

# Create random noisy strings
random_noise = generate_random_noise(num_sentences=100, length=15)

# Create contradictory sentences
contradictory_sentences = [
    "The sky is green and grass is red.",
    "Water is dry and fire is cold.",
    "The square is round and triangles have four sides.",
] * 33  # Repeat to match size

# Generate random sentences from a vocabulary
vocab = ["foo", "bar", "baz", "qux", "nonsense", "random"]
random_sentences = generate_random_sentences(vocab, length=10, num_sentences=100)

# Combine all synthetic OOD data
synthetic_ood_data = (
    shuffled_sentences
    + random_word_sentences
    + random_noise
    + contradictory_sentences
    + random_sentences
)

# Save synthetic OOD data
with open("synthetic_ood.txt", "w") as f:
    f.writelines("\n".join(synthetic_ood_data))

In [ ]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 18A3-4780

 Directory of c:\Users\ayush\OneDrive\Desktop\Deep Learning\Final Project

11/29/2024  11:18 PM    <DIR>          .
11/29/2024  05:22 PM    <DIR>          ..
11/29/2024  08:38 PM            33,918 final_proj.ipynb
11/29/2024  08:03 PM               674 id_data.pt
11/29/2024  08:06 PM       793,646,860 id_data_chunk_0.pt
11/29/2024  08:06 PM       793,646,860 id_data_chunk_1.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_10.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_11.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_12.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_13.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_14.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_15.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_16.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_17.pt
11/29/2024  08:06 PM       793,646,867 id_data_chunk_18.pt
11/29/2024  08:06 PM   

In [ ]:
import random
import json

# Select 1% of the dataset randomly while iterating
subset_size = 0.01  # Percentage of data to keep
arxiv_abstracts = []
dataset_path = "manual_data/arxiv-metadata-oai-snapshot.json"

with open(dataset_path, "r") as f:
    for line in f:
        if random.random() < subset_size:  # Randomly include lines
            record = json.loads(line)
            if "abstract" in record:
                arxiv_abstracts.append(record["abstract"])

print(f"Total abstracts in 1% subset: {len(arxiv_abstracts)}")

Total abstracts in 1% subset: 25977


In [ ]:
# Load synthetic data from file
with open("synthetic_ood.txt", "r") as f:
    synthetic_data = f.readlines()

# Combine synthetic and ArXiv data
ood_text_data = synthetic_data + arxiv_abstracts
print(f"Combined OOD dataset size: {len(ood_text_data)}")

Combined OOD dataset size: 26476


In [ ]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


# Tokenize OOD data
def tokenize_text_data(text_data, tokenizer):
    """
    Tokenize text data using the given tokenizer.
    """
    tokenized = tokenizer(text_data, padding=True, truncation=True, return_tensors="pt")
    return tokenized


ood_tokenized_data = tokenize_text_data(ood_text_data, tokenizer)

In [ ]:
import torch

# Save tokenized OOD data
torch.save(ood_tokenized_data, "ood_data.pt")

print("OOD data prepared and saved as 'ood_data.pt'.")

OOD data prepared and saved as 'ood_data.pt'.


In [ ]:
import torch
from glob import glob

# Load all ID data chunks
id_data_files = sorted(glob("id_data_chunk_*.pt"))  # List all chunk files in order
id_data = {"input_ids": [], "attention_mask": []}  # Initialize combined dataset

for file in id_data_files:
    chunk = torch.load(file)
    for key in id_data.keys():
        id_data[key].append(chunk[key])

# Combine all chunks into tensors
id_data["input_ids"] = torch.cat(id_data["input_ids"])
id_data["attention_mask"] = torch.cat(id_data["attention_mask"])

print(f"ID data shape: {id_data['input_ids'].shape}")

C:\Users\ayush\AppData\Local\Temp\ipykernel_28644\1015839721.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chunk = torch.load(file)


ID data shape: torch.Size([64587, 512])


In [ ]:
ood_data = torch.load("ood_data.pt")

print(f"OOD data shape: {ood_data['input_ids'].shape}")

C:\Users\ayush\AppData\Local\Temp\ipykernel_28644\3012189875.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ood_data = torch.load("ood_data.pt")


OOD data shape: torch.Size([26476, 512])


In [ ]:
id_labels = torch.zeros(id_data["input_ids"].shape[0])  # Label 0 for ID
ood_labels = torch.ones(ood_data["input_ids"].shape[0])  # Label 1 for OOD

In [ ]:
combined_data = {
    "input_ids": torch.cat([id_data["input_ids"], ood_data["input_ids"]]),
    "attention_mask": torch.cat(
        [id_data["attention_mask"], ood_data["attention_mask"]]
    ),
    "labels": torch.cat([id_labels, ood_labels]),
}

print(f"Combined data shape: {combined_data['input_ids'].shape}")
print(f"Combined labels shape: {combined_data['labels'].shape}")

Combined data shape: torch.Size([91063, 512])
Combined labels shape: torch.Size([91063])


In [ ]:
torch.save(combined_data, "combined_data.pt")
print("Combined data saved successfully as 'combined_data.pt'.")

Combined data saved successfully as 'combined_data.pt'.


In [ ]:
from google.colab import files

# Re-upload the combined_data.pt file
uploaded = files.upload()


Saving combined_data.zip to combined_data.zip


In [ ]:
!gzip -d combined_data.pt.gz

# Decompress ZIP file
!unzip combined_data.zip


gzip: combined_data.pt.gz: No such file or directory
Archive:  combined_data.zip
  inflating: combined_data.pt        


In [ ]:
import os

# Verify the file size
print(f"File size: {os.path.getsize('combined_data.pt')} bytes")


File size: 746354110 bytes


In [ ]:
import torch

# Load the combined data
combined_data = torch.load("combined_data.pt")


<ipython-input-6-88e53f68338e>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  combined_data = torch.load("combined_data.pt")


In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training (70%), validation (15%), and testing (15%)
train_size = 0.7
val_size = 0.15

# Train-test split
train_indices, temp_indices = train_test_split(
    range(combined_data["input_ids"].shape[0]),
    test_size=(1 - train_size),
    random_state=42,
)

# Validation-test split
val_indices, test_indices = train_test_split(
    temp_indices, test_size=(val_size / (1 - train_size)), random_state=42
)

# Create split datasets
train_data = {key: combined_data[key][train_indices] for key in combined_data.keys()}
val_data = {key: combined_data[key][val_indices] for key in combined_data.keys()}
test_data = {key: combined_data[key][test_indices] for key in combined_data.keys()}

print(f"Train data shape: {train_data['input_ids'].shape}")
print(f"Validation data shape: {val_data['input_ids'].shape}")
print(f"Test data shape: {test_data['input_ids'].shape}")

Train data shape: torch.Size([63744, 512])
Validation data shape: torch.Size([13659, 512])
Test data shape: torch.Size([13660, 512])


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.nn.functional import softmax

In [ ]:
train_dataset = TensorDataset(
    train_data["input_ids"], train_data["attention_mask"], train_data["labels"]
)
val_dataset = TensorDataset(
    val_data["input_ids"], val_data["attention_mask"], val_data["labels"]
)
test_dataset = TensorDataset(
    test_data["input_ids"], test_data["attention_mask"], test_data["labels"]
)

# Create DataLoaders
batch_size = 16  # Try reducing from 32 to 16 or even 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        # Ensure labels are correctly shaped and of the proper type
        labels = labels.view(-1).long()  # Convert labels to torch.long

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Compute loss explicitly
        loss = loss_fn(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/3, Loss: 0.0046626751642228756
